A little bit of segue on the previous notebook, I've recreated another jupyter to explore more on matrix factorization for collaborative filtering without the use of TF (did gone overboard with previous notebook)

Goal in mind being using numpy and scipy for mf svd or nmf with a little bit more control on matrix factorization for different federated clients as I'll explore later.

In [22]:
from src.data_examples.ex2_data_loader import ExampleDataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import svds as scipy_svds

pd.options.mode.chained_assignment = None

In [23]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [24]:
# to trucate any other features outside of matrix factorization
# select only user_id, movie_id, and rating for our matrix

features_X = ['user_id', 'movie_id']
feature_Y = 'rating'

df_main = data_loader.df[features_X + [feature_Y]]


for feature in features_X:
  df_main[feature] = df_main[feature].astype('int32')
# df_main['rating'] = df_main['rating'].apply(lambda x: (x*5) + 2)
df_main

,user_id,movie_id,rating
0,195,241,0.2
1,304,241,0.6
2,5,241,0.4
3,233,241,0.4
4,62,241,0.2
...,...,...,...
99995,862,1678,0.2
99996,862,1677,-0.2
99997,862,1679,0.0
99998,895,1680,0.2


In [25]:
def df_to_np_sparse_matrix(df, U_c, V_c, val_c):
  # pivot the data with movie_id as column to create sparse mxn matrix 
  df = df[[U_c, V_c, val_c]]
  df_pivot = df.pivot(index=U_c, columns=V_c, values=val_c).fillna(0)
  return df_pivot.values

def sparse_matrix_normalize(mat_sparse):
  # normalize matrix from means of axis 1
  mat_mean = np.mean(mat_sparse, axis=1)
  return (mat_sparse - mat_mean.reshape(-1, 1)), mat_mean

def eval_svd_factorization(df, U_c, V_c, val_c, latent_factor=25):
  mat_sparse = df_to_np_sparse_matrix(df_main, 'user_id', 'movie_id', 'rating')
  mat_sparse_norm, mat_sparse_mean = sparse_matrix_normalize(mat_sparse)

  U, sigma, Vt = scipy_svds(mat_sparse, k=latent_factor)
  sigma = np.diag(sigma)

  mat_prediction = np.dot(np.dot(U, sigma), Vt) + mat_sparse_mean.reshape(-1, 1)

  return mat_prediction, (U, sigma, Vt)


In [26]:
mat_pred, (U, sigma, Vt) = eval_svd_factorization(df_main, 'user_id', 'movie_id', 'rating', latent_factor=25)

df_pred = pd.DataFrame(mat_pred) 
df_pred # which will directly encodes user_id as index axis 1 and movie_id as columns

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.591173,0.218883,0.216366,0.320632,0.074675,0.128291,0.496458,0.195799,0.381042,0.227432,...,0.050663,0.053200,0.050407,0.052081,0.052497,0.052149,0.052012,0.052081,0.054034,0.059397
1,0.239390,-0.008896,-0.025523,0.045449,0.015504,0.047360,0.121989,0.106183,0.207451,0.064969,...,0.012884,0.010631,0.013449,0.012604,0.011902,0.011734,0.013475,0.012604,0.013509,0.010338
2,0.042691,0.010405,0.026525,-0.016571,-0.013632,0.002582,-0.035287,-0.015781,-0.047577,0.009940,...,0.004925,0.003757,0.006038,0.005113,0.004857,0.003596,0.006630,0.005113,0.005093,0.005625
3,0.064871,-0.016956,0.025028,-0.014872,0.017362,0.010358,0.062990,-0.010724,0.058845,-0.002500,...,0.006617,0.005249,0.006122,0.006659,0.007138,0.005123,0.008194,0.006659,0.006491,0.006808
4,0.395072,0.085839,0.024492,0.187126,0.004489,0.000817,0.323427,0.142048,-0.114113,0.044969,...,0.017919,0.018186,0.017251,0.018193,0.016831,0.018249,0.018136,0.018193,0.019005,0.017480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.339086,0.015353,0.011655,0.019144,0.056034,0.024695,0.145706,0.102731,0.168634,0.023343,...,0.014449,0.012582,0.014152,0.013199,0.013107,0.014511,0.011886,0.013199,0.013769,0.011267
939,0.044876,0.006287,-0.008784,0.220981,0.060224,0.013593,0.296255,0.238947,0.194064,0.010604,...,0.018983,0.023172,0.018715,0.018549,0.019140,0.017507,0.019591,0.018549,0.018872,0.017431
940,0.262788,0.015168,0.027576,0.006428,-0.001998,0.005935,0.189606,0.089612,0.125642,0.024956,...,0.004565,0.003910,0.005605,0.005351,0.006312,0.005493,0.005208,0.005351,0.006143,0.005221
941,0.135355,0.030077,-0.000674,0.020363,0.003278,0.003302,0.000714,0.173069,0.007655,0.024929,...,0.022885,0.023883,0.022420,0.021284,0.021475,0.019655,0.022914,0.021284,0.022923,0.020821


In [27]:
data_loader.df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         100000 non-null  object 
 1   movie_id        100000 non-null  object 
 2   rating          100000 non-null  float64
 3   unix_timestamp  100000 non-null  uint64 
dtypes: float64(1), object(2), uint64(1)
memory usage: 3.1+ MB


In [28]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   100000 non-null  int32  
 1   movie_id  100000 non-null  int32  
 2   rating    100000 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 2.3 MB


In [37]:
tgt_user_id = 420
n_movies = 15

pred_movie_ids = df_pred.iloc[tgt_user_id].sort_values(ascending=False)[:n_movies].index


__fmt_movie = lambda movie_id: f'movie_id: {movie_id}'.ljust(17) + data_loader.df_movies.loc[movie_id]['title']

pred_movie_ids = df_main[df_main['user_id'] == tgt_user_id]['movie_id'] [:n_movies]
print('recommendtion')
for movie_id in pred_movie_ids:
  print(__fmt_movie(movie_id))



recommendtion
movie_id: 237    Raising Arizona (1987)
movie_id: 172    Princess Bride, The (1987)
movie_id: 268    Full Monty, The (1997)
movie_id: 301    L.A. Confidential (1997)
movie_id: 332    Game, The (1997)
movie_id: 116    Rock, The (1996)
movie_id: 99     Fargo (1996)
movie_id: 78     Fugitive, The (1993)
movie_id: 97     Silence of the Lambs, The (1991)
movie_id: 55     Pulp Fiction (1994)
movie_id: 11     Usual Suspects, The (1995)
movie_id: 330    Edge, The (1997)
movie_id: 95     Terminator 2: Judgment Day (1991)
movie_id: 175    Aliens (1986)
movie_id: 207    Young Frankenstein (1974)
